# SD

In [ ]:
from clip_slider_pipeline import CLIPSlider
from diffusers import StableDiffusionPipeline
import torch
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to(torch.float16)
slider = CLIPSlider(sd_pipe=pipe, 
                    device=torch.device("cuda:0" if torch.cuda.is_available()),
                    target_word="happy",
                    opposite="sad")

In [ ]:
scale = 2
slider.spectrum(prompt="a house",low_scale=-scale, high_scale=scale, steps=5, seed=0)

# SDXL

In [ ]:
from clip_slider_pipeline import CLIPSliderXL
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
import torch
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, vae=vae,
)
slider_xl = CLIPSliderXL(sd_pipe=pipe, 
                    device=torch.device("cuda:0" if torch.cuda.is_available()),
                    target_word="happy",
                    opposite="sad")

In [ ]:
scale = 2
slider_xl.spectrum(prompt="a house",low_scale=-scale, high_scale=scale, steps=5, seed=0)

# SDXL Flash

In [ ]:
import torch
from clip_slider_pipeline import CLIPSliderXL
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from diffusers.utils import make_image_grid

pipe = StableDiffusionXLPipeline.from_pretrained("sd-community/sdxl-flash").to("cuda", torch.float16)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

flash_slider = CLIPSliderXL(sd_pipe=pipe, 
                    device=torch.device("cuda"),
                    target_word="happy",
                    opposite="sad")

In [ ]:
scale = 2
flash_slider.spectrum(prompt="a house",low_scale=-scale, high_scale=scale, steps=5, seed=0)

# SDXL & IP Adapter

In [ ]:
import torch
from clip_slider_pipeline import CLIPSliderXL
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from diffusers.utils import make_image_grid, load_image

pipe = StableDiffusionXLPipeline.from_pretrained("sd-community/sdxl-flash").to("cuda", torch.float16)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

# load IP Adapter
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
scale = 0.8
pipe.set_ip_adapter_scale(scale)

flash_slider = CLIPSliderXL(sd_pipe=pipe, 
                    device=torch.device("cuda"),
                    target_word="happy",
                    opposite="sad")



In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-sdxl-init.png"
image = load_image(url)

scale = 2
grid = flash_slider.spectrum(prompt="a house",low_scale=-scale, high_scale=scale, steps=5,
                             ip_adapter_image=image, 
                             num_inference_steps=10,
                             seed=0)

# SDXL ControlNet

In [ ]:
import torch
import random
from clip_slider_pipeline import CLIPSliderXL
from tqdm import tqdm
from controlnet_aux import PidiNetDetector, HEDdetector
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers.utils import make_image_grid, load_image
import numpy as np
import cv2
from PIL import Image

controlnet = ControlNetModel.from_pretrained(
    "xinsir/controlnet-scribble-sdxl-1.0", # insert here your choice of controlnet
    torch_dtype=torch.float16
)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "sd-community/sdxl-flash",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
)

flash_slider = CLIPSliderXL(sd_pipe=pipe, 
                    device=torch.device("cuda"),
                    target_word="happy",
                    opposite="sad")

processor = HEDdetector.from_pretrained('lllyasviel/Annotators')
def nms(x, t, s):
    x = cv2.GaussianBlur(x.astype(np.float32), (0, 0), s)

    f1 = np.array([[0, 0, 0], [1, 1, 1], [0, 0, 0]], dtype=np.uint8)
    f2 = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]], dtype=np.uint8)
    f3 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=np.uint8)
    f4 = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]], dtype=np.uint8)

    y = np.zeros_like(x)

    for f in [f1, f2, f3, f4]:
        np.putmask(y, cv2.dilate(x, kernel=f) == x, x)

    z = np.zeros_like(y, dtype=np.uint8)
    z[y > t] = 255
    return z


In [ ]:
image = load_image("flower.jpg")
controlnet_img = processor(image, scribble=False)
# following is some processing to simulate human sketch draw, different threshold can generate different width of lines
controlnet_img = np.array(controlnet_img)
controlnet_img = nms(controlnet_img, 127, 3)
controlnet_img = cv2.GaussianBlur(controlnet_img, (0, 0), 3)

# higher threshold, thiner line
random_val = int(round(random.uniform(0.01, 0.10), 2) * 255)
controlnet_img[controlnet_img > random_val] = 255
controlnet_img[controlnet_img < 255] = 0
image = Image.fromarray(controlnet_img)

scale = 2
grid = flash_slider.spectrum(prompt="a house",low_scale=-scale, high_scale=scale, steps=5,
                             image=image, 
                             controlnet_conditioning_scale = 0.5,
                             num_inference_steps=8,
                             seed=0)

# SD 3

In [ ]:
import torch
from diffusers import , StableDiffusion3Pipeline
from clip_slider_pipeline  import CLIPSlider3

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16)
slider3 = CLIPSlider3(sd_pipe=pipe, 
                    device=torch.device("cuda"),
                    target_word="happy",
                    opposite="sad")

In [ ]:
slider3(prompt="a house",low_scale=-scale, high_scale=scale, steps=5, seed=0)